# Georgia LDU Closure Analysis (2012-2016)

In [1]:
from helper import *
show_dfs = False

## Datasets

### Patients

- Data from the *Emory MCH Linked Vital Records Data Repository* (private data source) is used to identify per-patient birth data for births in 2011 by birthing LDU, payor status, race, ethnicity, and county of residence.

In [2]:
# Load patient data.
patients = pd.read_csv('data/patients.csv')

# Drop irrelevant columns.
del patients['Ethnicity']

show_df(patients, show_dfs)

### Labor & Delivery Units

- Data from *Georgia Maternal and Infant Health Research Group (GMIHRG)* (private data source) is used to identify the LDUs of interest and their birth counts in 2008, 2011, and 2012; numbers of OBs, FPs, and CNMs in 2011 and 2016; and average ages of OBs in 2011 and 2016.
- Data from the *Emory MCH Linked Vital Records Data Repository* (private data source) is used to obtain 2001 and 2011 number of births per-LDU to residents and non-residents of the county the LDU is in. It is also the source of LDU names that we consider standard.
- Data from the [*U.S. Census Bureau*]() is used to identify urban areas in 2010.
- Data from [*Google Maps*](https://www.google.com/maps/d/u/0/edit?mid=1_xMZrJgPbcInCcq8CgdmwuncWMWSOoJj&usp=sharing) is used to identify, for each LDU, the closest (other) LDU (within Georgia), the number of driving miles to the closest LDU, the closest urban area (in any state), and the number of driving miles to the closest urban area in 2011.

This analysis primarily focuses on 2011, so all columns irrelevant to the quantitative analysis are dropped.

In [3]:
# Load LDU data.
ldus = pd.read_csv('data/ldus.csv')

# Drop irrelevant columns.
for col in ['Lat/Long', '# Births (2008)', '# Births (2012)', '# OBs (2016)', \
            'Ave. OB Age (2016)', '# FPs (2016)', '# CNMs (2016)', \
            '# County Res. Births (2001)', '# County Non-Res. Births (2001)', \
            '# County Res. Births (2011)', '# County Non-Res. Births (2011)']:
    del ldus[col]

# Drop '(2011)' from column names since we now only have 2011 data.
ldus.rename(columns={x: x.replace(' (2011)', '') for x in list(ldus.head())}, \
            inplace=True)

show_df(ldus, show_dfs)

### Regional Data (Counties and PCSAs)

- Data from [*OASIS*](https://oasis.state.ga.us) is used to obtain birth and population counts in 2001 and 2011 by county.
- Data from the [*Office of Management and Budget (OMB)*](https://obamawhitehouse.archives.gov/sites/default/files/omb/bulletins/2013/b-13-01.pdf) is used to identify counties contained in the Atlanta-Sandy Springs-Roswell Metropolitan Statistical Area (MSA) based on the 2010 Census (see page 23).
- Data from the [*U.S. Census Bureau*](https://data.census.gov/cedsci/table?q=&t=Income%20and%20Poverty&g=0400000US13%240500000&y=2011&tid=ACSST5Y2011.S1903) is used to obtain 2011 median household income by county.
- Data from the [*Georgia Board of Health Care Workforce*](https://healthcareworkforce.georgia.gov/basic-physician-needs-reports-pcsa-primary-care-service-area) in the year 2008 is used to map counties to PCSAs.

In [4]:
# Load county data.
counties = pd.read_csv('data/counties.csv')

# Drop '(2011)' from column names since they are default.
counties.rename(columns={x: x.replace(' (2011)', '') \
                         for x in list(counties.head())}, inplace=True)

show_df(counties, show_dfs)

## Determining the Sample

### Inclusion Criteria for Rural PCSAs

PCSAs included in the sample are *rural*, meaning that in 2011:

1. They did not contain any counties that were within the Atlanta MSA.
2. They did not contain any counties with population at least 50,000.
3. They contained exactly one LDU.

In [5]:
# Construct a DataFrame of 96 PCSAs.
pcsas = pd.DataFrame({'PCSA' : [x+1 for x in range(96)]})

# Identify PCSAs that have no counties in the Atlanta MSA.
df = (counties.groupby('PCSA')['In MSA (2010)'].sum() == 0).to_frame('Inc. MSA')
df1 = pcsas.join(df, on='PCSA')

# Identify PCSAs whose counties all have population strictly less than 50K.
df = (counties.groupby('PCSA')['Population'].max() < 50000).to_frame('Inc. Pop')
df2 = pcsas.join(df, on='PCSA')

# Identify PCSAs containing exactly one LDU.
df = ldus.groupby('County').size().to_frame('# LDUs')
df = counties.join(df, on='County')
df = (df.groupby('PCSA')['# LDUs'].sum() == 1).to_frame('Inc. 1 LDU')
df3 = pcsas.join(df, on='PCSA')

# Determine which PCSAs are in sample.
pcsas['In Sample'] = df1['Inc. MSA'] & df2['Inc. Pop'] & df3['Inc. 1 LDU']

show_df(pcsas, show_dfs)

### Narrowing Patients, LDUs, and Counties to the Sample

With the 30 PCSAs in sample identified, the three other datasets can be winnowed down. All calculations from here on out can safely assume, due to Inclusion Criteria 3, that there is a 1:1 correspondence between LDUs and PCSAs.

In [6]:
# Collect the counties that are in sample.
df1 = counties.join(pcsas.set_index('PCSA'), on='PCSA')
s_counties = df1.drop(df1[df1['In Sample'].map(lambda x: not x)].index)
del s_counties['In Sample']

# Collect the LDUs that are in sample.
df2 = ldus.join(df1[['County', 'In Sample']].set_index('County'), on='County')
s_ldus = df2.drop(df2[df2['In Sample'].map(lambda x: not x)].index)
del s_ldus['In Sample']

# Collect the patients that are in sample.
df3 = patients.join(df2[['LDU', 'In Sample']].set_index('LDU'), on='LDU')
s_patients = df3.drop(df3[df3['In Sample'].map(lambda x: not x)].index)
del s_patients['In Sample']

# Finally, collect the PCSAs that are in sample.
s_pcsas = pcsas.drop(pcsas[pcsas['In Sample'].map(lambda x: not x)].index)
del s_pcsas['In Sample']

## Derived Columns

Based on the raw data above, we derive a series of new columns at the patient, LDU, and PCSA levels.

### Patient Payor Types/Groups and Residence

We aggregate different payor statuses into types and groups according to the dictionaries below. We also identify which patients gave birth to an LDU within their county and PCSA of residence.

In [7]:
# Assign patients their payor types.
ptypes = {'Unknown': 'Other/Unknown',
          'Champus': 'Commercial/Employer-Based',
          'Medicaid': 'Medicaid',
          'Commercial Insurance': 'Commercial/Employer-Based',
          'Other Government Assistance': 'Other Govt.',
          'Other': 'Other/Unknown',
          'Self Pay': 'Self Pay'}
s_patients['Payor Type'] = s_patients['Payor'].map(lambda x: ptypes[x])

# Assign patients their payor groups.
pgroups = {'Commercial/Employer-Based': 'Commercial/Employer-Based',
           'Medicaid': 'Assistance/Self Pay',
           'Other Govt.': 'Assistance/Self Pay',
           'Self Pay': 'Assistance/Self Pay',
           'Other/Unknown': 'Other/Unknown'}
s_patients['Payor Group'] = s_patients['Payor Type'].map(lambda x: pgroups[x])

# Use the counties of each LDU to identify whether patients gave birth in their
# county of residence.
df1 = s_patients.join(ldus[['LDU', 'County']].set_index('LDU'), on='LDU')
s_patients['In Res. County'] = s_patients['Res. County'] == df1['County']

# Use the counties of each LDU and the mappings of counties to PCSAs to identify
# whether patients gave birth in their PCSA of residence.
df2 = s_patients.join(counties[['County', 'PCSA']].set_index('County'), \
                      on='Res. County')
df3 = df1.join(counties[['County', 'PCSA']].set_index('County'), on='County')
s_patients['In Res. PCSA'] = df2['PCSA'] == df3['PCSA']

show_df(s_patients, show_dfs)

### Patient Demographics, Payor Types/Groups, and Resident Births by LDU

Using the 2011 patient-level data in the above table, we aggregate the following measures by LDU:
- Proportion (%) of patients by race (Black, white, and other)
- Proportion (%) of patients by payor type (commercial/employer-based, Medicaid, self pay, other government, and other/unknown)
- Number of patients by payor group (commercial/employer-based and assistance/self pay)
- Number of patients in each pairwise intersection of Black vs. white and commercial/employer-based vs. assistance/self pay
- Number and proportion (%) of births to residents and non-residents of the LDU's county and PCSA

In [8]:
# Count total number of patients per LDU.
df1 = s_patients.groupby('LDU').size().to_frame('# Patients')
s_ldus = s_ldus.join(df1, on='LDU')

# Calculate #s and %s of Black, white, and other patients per LDU.
df1 = s_patients.groupby(['LDU', 'Race']).size().to_frame('#').reset_index()
for race in ['Black or African-American', 'White']:
    df2 = df1[df1['Race'] == race]
    df2 = s_ldus.join(df2[['LDU', '#']].set_index('LDU'), on='LDU')
    s_ldus['# '+race.split()[0]+' Patients'] = df2['#']
    s_ldus['% '+race.split()[0]+' Patients'] = 100*df2['#']/s_ldus['# Patients']
s_ldus['# Other/Unknown Race'] = s_ldus['# Patients'] \
                                 - s_ldus['# Black Patients'] \
                                 - s_ldus['# White Patients']
s_ldus['% Other/Unknown Race'] = 100 * s_ldus['# Other/Unknown Race'] \
                                 / s_ldus['# Patients']

# Calculate #s and %s of patients per payor type per LDU.
df1 = s_patients.groupby(['LDU', 'Payor Type']).size().to_frame('#').reset_index()
for ptype in ['Commercial/Employer-Based', 'Medicaid', 'Self Pay', \
              'Other Govt.', 'Other/Unknown']:
    df2 = df1[df1['Payor Type'] == ptype]
    df2 = s_ldus.join(df2[['LDU', '#']].set_index('LDU'), on='LDU')
    s_ldus['# '+ptype+' Payors'] = df2['#'].fillna(0)
    s_ldus['% '+ptype+' Payors'] = 100*df2['#'].fillna(0) / s_ldus['# Patients']

# Count patients per payor group per LDU.
df1 = s_patients.groupby(['LDU', 'Payor Group']).size().to_frame('#').reset_index()
for pgroup in ['Commercial/Employer-Based', 'Assistance/Self Pay']:
    df2 = df1[df1['Payor Group'] == pgroup]
    df2 = s_ldus.join(df2[['LDU', '#']].set_index('LDU'), on='LDU')
    s_ldus['# ' + pgroup + ' Payors'] = df2['#'].fillna(0)
    
# Count patients by intersection of race and payor groups.
df1 = s_patients.groupby(['LDU', 'Race', 'Payor Group']).size()\
                .to_frame('#').reset_index()
for race, pgroup in itertools.product(['Black or African-American', 'White'], \
                                      ['Commercial/Employer-Based', 'Assistance/Self Pay']):
    df2 = df1[(df1['Race'] == race) & (df1['Payor Group'] == pgroup)]
    df2 = s_ldus.join(df2[['LDU', '#']].set_index('LDU'), on='LDU')
    s_ldus['# ' + race.split()[0] + ' ' + pgroup] = df2['#'].fillna(0)

# Count numbers and calculate %s of births to county and PCSA residents.
for area in ['County', 'PCSA']:
    res_str = area + ' Res. Patients'
    nonres_str = area + ' Non-Res. Patients'
    df1 = s_patients.groupby(['LDU', 'In Res. ' + area]).size()\
                    .to_frame('# '+res_str).reset_index()
    df2 = df1[df1['In Res. ' + area]]
    s_ldus = s_ldus.join(df2[['LDU', '# '+res_str]].set_index('LDU'), on='LDU')
    s_ldus['% '+res_str] = 100 * s_ldus['# '+res_str] / s_ldus['# Patients']
    s_ldus['# '+nonres_str] = s_ldus['# Patients'] - s_ldus['# '+res_str]
    s_ldus['% '+nonres_str] = 100 - s_ldus['% '+res_str]

show_df(s_ldus, show_dfs)

### Provider Count and Load by LDU

We count providers by the number of OB equivalents per LDU in 2011 and use it to calculate the number of births per provider. An OB equivalent is calculated as:
$$
(\#OBs) + \frac{1}{1.55} \cdot (\#CNMs) + \frac{0.7}{1.55} \cdot (\#FPs)
$$

We also identify the birth volume of the closest LDU.

In [9]:
# Calculate OB equivalents and births per provider.
s_ldus['OB Equiv.'] = s_ldus['# OBs'] + s_ldus['# CNMs'] / 1.55 \
                      + (0.7/1.55) * s_ldus['# FPs']
s_ldus['Births per Provider'] = s_ldus['# Births'] / s_ldus['OB Equiv.']

# Get the 2011 birth volume at the closest LDU.
s_ldus = s_ldus.join(ldus[['LDU', '# Births']].set_index('LDU'), \
                     on='Closest GA LDU', rsuffix=' at Closest GA LDU')

show_df(s_ldus, show_dfs)

### County Birth Volume and Population Demographics by PCSA

We additionally calculate, per-PCSA, the aggregate number of births (2001 and 2011), population (2001 and 2011), female population (2011), % Black female population (2011), % white female population (2011), % other female population (2011), and household income (2011). Median household income is available on a per-county basis; to calculate a PCSA's household income, we take a weighted average of its counties' median household incomes weighted by each county's proportion of the PCSA population. Mathematically, for a PCSA $p$ containing counties $c_1, \ldots, c_k$ we have:
$$
\text{income}(p) = \sum_{i=1}^k \left(\frac{\text{population}(c_i)}{\text{population}(p)}\right) \cdot \text{income}(c_i) = \frac{1}{\text{population}(p)} \cdot \sum_{i=1}^k \text{population}(c_i) \cdot \text{income}(c_i)
$$

In [10]:
# Calculate the number of births per-PCSA in 2001 and 2011, the total population
# per-PCSA in 2001 and 2011, and the female populations per-PCSA in 2011.
for m in ['# Births (2001)', '# Births', 'Population (2001)', 'Population', \
          'Females 15-44']:
    s_pcsas = s_pcsas.join(s_counties.groupby('PCSA')[m].sum().to_frame(m), \
                           on='PCSA')
    
# Calculate the #s and %s of Black, white, and other race females by PCSA.
for m in ['Black Females 15-44', 'White Females 15-44']:
    df = s_counties.groupby('PCSA')[m].sum().to_frame('#')
    df = s_pcsas.join(df, on='PCSA')
    s_pcsas[m] = df['#']
    s_pcsas['% ' + m] = 100 * df['#'] / s_pcsas['Females 15-44']
s_pcsas['Other Females 15-44'] = s_pcsas['Females 15-44'] \
                                 - s_pcsas['Black Females 15-44'] \
                                 - s_pcsas['White Females 15-44']
s_pcsas['% Other Females 15-44'] = 100 * s_pcsas['Other Females 15-44'] \
                                   / s_pcsas['Females 15-44']
    
# Calculate the median household income per-PCSA using population-weighted
# proportions by county.
df = s_counties.groupby('PCSA')\
               .apply(lambda x: (x['Population']*x['Median Household Income'])\
               .sum()).to_frame('incprod')
df = s_pcsas.join(df, on='PCSA')
s_pcsas['Household Income'] = df['incprod'] / df['Population']

Finally, we identify which PCSAs had LDUs that closed (recall the assumption of 1:1 LDU:PCSA correspondence by Inclusion Criterium 3).

In [11]:
close_str = 'Closed 2012-2016'
df = s_ldus.groupby('County')[close_str].sum().to_frame(close_str)
df = s_counties[['County', 'PCSA']].join(df, on='County').fillna(0)
df = df.groupby('PCSA')[close_str].sum().to_frame(close_str)
s_pcsas = s_pcsas.join(df, on='PCSA')

show_df(s_pcsas, show_dfs)

### Dumping the Processed Data

We write out the processed patient, LDU, county, and PCSA data for easier inspection.

In [12]:
s_pcsas.to_csv('data/processed_pcsas.csv')
s_counties.to_csv('data/processed_counties.csv')
s_ldus.to_csv('data/processed_ldus.csv')
s_patients.to_csv('data/processed_patients.csv')

## Analysis

There are four classes of measures $m$ that we report statistics on.

1. *Counts* (e.g., the number of births in a given year, household incomes, or populations), for which we report the total $\sum_{p \in P} m(p)$ and, for each subset of $P^{open}, P^{closed} \subseteq P$, the:
    - Mean: $mean(\{m(p) : p \in P^{open/closed}\})$
    - Median: $median(\{m(p) : p \in P^{open/closed}\})$
    - Min: $min(\{m(p) : p \in P^{open/closed}\})$
    - Max: $max(\{m(p) : p \in P^{open/closed}\})$
    - p-value of an [independent 2-sample t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html#scipy.stats.ttest_ind) on $\{m(p) : p \in P^{open}\}$ vs. $\{m(p) : p \in P^{closed}\}$
    - p-value of a [Mann-Whitney U rank test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html#scipy.stats.mannwhitneyu) on $\{m(p) : p \in P^{open}\}$ vs. $\{m(p) : p \in P^{closed}\}$
2. *Proportions* (e.g., the percentage of the female population in a given PCSA that is Black), for which we report the same statistics as for Counts (but the total remains the raw total, not the sum of the proportions).
3. *Odds Ratios* (e.g., the odds of being a patient with Commercial/Employer-Based insurance in a PCSA with a closed LDU compared to one with Assistance/Self Pay), for which we perform a [Fisher exact test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.fisher_exact.html#scipy.stats.fisher_exact). The exact details depend on the test. In some scenarios where confounding variables are investigated, [Cochran-Mantel-Haenszel ratios](https://openepi.com/TwobyTwo/TwobyTwo.htm) are calculated for the stratified data.
4. *Basics* (i.e., a simple count or percentage without any major statistics), for which we report the measure aggregated by open/closed LDUs/PCSAs and the percentage of the whole in each category.

### Table 1: PCSA Population Demographics by LDU Closure Status (2011)

Each PCSA's total 2001 and 2011 population ("Population (2001)" and "Population"), 2011 female population ("Females 15-44"), and 2011 household income ("Household Income") are investigated as Counts.

Each PCSA's 2011 Black female population ("% Black Females 15-44"), white female population ("% White Females 15-44"), and other race female population ("% Other Females 15-44") are investigated as Proportions.

As an example, "% Black Females 15-44" operates on the set $\left\{\frac{\text{blkfem}_{2011}(p)}{\text{fem}_{2011}(p)} : p \in P\right\}$.

In [13]:
for m in ['Population (2001)', 'Population', 'Females 15-44', \
          'Black Females 15-44', '% Black Females 15-44', \
          'White Females 15-44', '% White Females 15-44', \
          'Other Females 15-44', '% Other Females 15-44', 'Household Income']:
    statsbyclosure(s_pcsas, m)
    print()

Population (2001)
-----------------
Total:		866,984.000
Open:		mean=30,720.375	median=26,732.000	(9,953.000-52,018.000)
Closed:		mean=21,615.833	median=21,917.500	(15,885.000-29,938.000)
2-Samp t-Test:	pval=0.130990
Mann-Whit Test:	pval=0.229693

Population
----------
Total:		935,890.000
Open:		mean=33,048.208	median=26,827.000	(9,679.000-61,530.000)
Closed:		mean=23,788.833	median=23,035.500	(17,125.000-31,086.000)
2-Samp t-Test:	pval=0.163305
Mann-Whit Test:	pval=0.250634

Females 15-44
-------------
Total:		178,044.000
Open:		mean=6,299.167	median=5,053.500	(1,646.000-12,226.000)
Closed:		mean=4,477.333	median=4,397.500	(3,281.000-6,405.000)
2-Samp t-Test:	pval=0.189474
Mann-Whit Test:	pval=0.296230

Black Females 15-44
-------------------
Total:		53,820.000
Open:		mean=1,888.583	median=1,737.000	(27.000-4,302.000)
Closed:		mean=1,415.667	median=1,310.500	(98.000-2,625.000)
2-Samp t-Test:	pval=0.432607
Mann-Whit Test:	pval=0.704705

% Black Females 15-44
---------------------
Total:

### Table 2: Patient Demographics and Insurance by LDU Closure Status (2011)

#### Patient Race

Each LDU's Black patients ("% Black Patients"), white patients ("% White Patients"), and other race patients ("% Other/Unknown Race") are investigated as Proportions.

As an example, "% Black Patients" operates on the set $\left\{\frac{|\{i \in \ell ~:~ \text{race}(i) = \text{black}\}|}{|\ell|} : \ell \in L\right\}$.


In [14]:
for m in ['% Black Patients', '% White Patients', '% Other/Unknown Race']:
    statsbyclosure(s_ldus, m)
    print()

% Black Patients
----------------
Total:		1,025.967
Open:		mean=32.617	median=31.851	(0.392-66.667)
Closed:		mean=40.528	median=41.918	(1.463-72.487)
2-Samp t-Test:	pval=0.401079
Mann-Whit Test:	pval=0.402129

% White Patients
----------------
Total:		1,608.736
Open:		mean=54.591	median=56.038	(22.180-94.828)
Closed:		mean=49.759	median=40.285	(25.397-95.122)
2-Samp t-Test:	pval=0.613088
Mann-Whit Test:	pval=0.493941

% Other/Unknown Race
--------------------
Total:		365.297
Open:		mean=12.792	median=9.444	(3.072-45.113)
Closed:		mean=9.713	median=3.844	(2.116-27.193)
2-Samp t-Test:	pval=0.541145
Mann-Whit Test:	pval=0.173999



An odds ratio for Black and white patients with known payor statuses is investigated against LDU closure status. For each $(r, s) \in \{\text{Black}, \text{White}\} \times \{\text{Open}, \text{Closed}\}$, we compute $\sum_{\ell \in L^{s}}|\{i \in \ell : (\text{race}(i) = r) \wedge (\text{payor}(i) \neq \text{Unknown})\}|$.

In [15]:
# TODO

To check if payor group is confounding, we further partition these black and white patients counts by payor groups "Assistance/Self Pay" and "Commercial/Employer-Based".

In [16]:
# TODO

#### Patient Insurance

Each LDU's breakdown of patients by payor type ("% Commercial/Employer-Based Payors", "% Medicaid Payors", "% Self Pay Payors", "% Other Govt. Payors", and "% Other/Unknown Payors") are investigated as Proportions.

As an example, "% Medicaid Payors" operates on the set $\left\{\frac{|\{i \in \ell ~:~ \text{payortype}(i) = \text{Medicaid}\}|}{|\ell|} : \ell \in L\right\}$.

In [17]:
for ptype in ['Commercial/Employer-Based', 'Medicaid', 'Self Pay', \
              'Other Govt.', 'Other/Unknown']:
    m = '% ' + ptype + ' Payors'
    statsbyclosure(s_ldus, m)
    print()

% Commercial/Employer-Based Payors
----------------------------------
Total:		373.717
Open:		mean=13.440	median=10.798	(0.196-30.508)
Closed:		mean=8.526	median=5.264	(3.285-25.366)
2-Samp t-Test:	pval=0.228103
Mann-Whit Test:	pval=0.157774

% Medicaid Payors
-----------------
Total:		1,787.220
Open:		mean=57.982	median=71.435	(5.501-91.892)
Closed:		mean=65.941	median=62.939	(29.268-94.545)
2-Samp t-Test:	pval=0.521442
Mann-Whit Test:	pval=0.781195

% Self Pay Payors
-----------------
Total:		238.819
Open:		mean=7.801	median=4.447	(0.000-39.850)
Closed:		mean=8.601	median=4.343	(0.000-26.496)
2-Samp t-Test:	pval=0.856446
Mann-Whit Test:	pval=0.979853

% Other Govt. Payors
--------------------
Total:		195.927
Open:		mean=7.985	median=0.343	(0.000-62.069)
Closed:		mean=0.714	median=0.427	(0.000-2.555)
2-Samp t-Test:	pval=0.347395
Mann-Whit Test:	pval=0.781195

% Other/Unknown Payors
----------------------
Total:		404.316
Open:		mean=12.792	median=1.375	(0.000-86.531)
Closed:		mean=16.21

An odds ratio for Assistance/Self Pay and Commercial/Employer-Based patient payor groups is investigated against LDU closure status. For each $(r, s) \in \{\text{Assistance/Self Pay}, \text{Commercial/Employer-Based}\} \times \{\text{Open}, \text{Closed}\}$, we compute $\sum_{\ell \in L^{s}}|\{i \in \ell : \text{payorgroup}(i) = r\}|$.

### Table 3: Birth Volume and Location by LDU Closure Status (2011)

**TODO**

In [19]:
# TODO

### Table 4: Obstetric Providers by LDU Closure Status

Each LDU's providers expressed as the number of OBs ("# OBs"), FPs ("# FPs"), CNMs ("# CNMs"), and OB eqivalents ("OB Equiv.") as well as the birth volume per provider ("Births per Provider") and average OB age ("Ave. OB Age") are investigated as Counts.

In [20]:
for m in ['# OBs', '# FPs', '# CNMs', 'OB Equiv.', 'Births per Provider', \
          'Ave. OB Age']:
    statsbyclosure(s_ldus, m)
    print()

# OBs
-----
Total:		79.000
Open:		mean=2.917	median=3.000	(0.000-6.000)
Closed:		mean=1.500	median=1.500	(1.000-2.000)
2-Samp t-Test:	pval=0.051751
Mann-Whit Test:	pval=0.043788

# FPs
-----
Total:		11.000
Open:		mean=0.333	median=0.000	(0.000-4.000)
Closed:		mean=0.500	median=0.000	(0.000-2.000)
2-Samp t-Test:	pval=0.701147
Mann-Whit Test:	pval=0.526705

# CNMs
------
Total:		15.000
Open:		mean=0.542	median=0.000	(0.000-4.000)
Closed:		mean=0.333	median=0.000	(0.000-1.000)
2-Samp t-Test:	pval=0.634779
Mann-Whit Test:	pval=0.939656

OB Equiv.
---------
Total:		93.645
Open:		mean=3.417	median=3.000	(1.000-7.935)
Closed:		mean=1.941	median=2.000	(1.645-2.097)
2-Samp t-Test:	pval=0.058615
Mann-Whit Test:	pval=0.043788

Births per Provider
-------------------
Total:		3,974.626
Open:		mean=141.187	median=133.638	(55.500-233.000)
Closed:		mean=97.691	median=108.250	(55.000-143.966)
2-Samp t-Test:	pval=0.029776
Mann-Whit Test:	pval=0.033120

Ave. OB Age
-----------
Total:		1,252.800
Open:		me